<a href="https://colab.research.google.com/github/Sapphirevic/MLOp/blob/main/duration_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -V

Python 3.10.12


In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
import os
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [3]:
from google.colab import files
upload = files.upload()

Saving green_tripdata_2021-01.parquet to green_tripdata_2021-01.parquet


In [4]:
!pip install mlflow #--quiet
!pip install pyngrok  #ngrok helps integrate mlflow in colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.6 MB/s eta 0:00:00


In [5]:
import mlflow
import subprocess
from pyngrok import ngrok, conf
import getpass

In [6]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('Learn MLFLOW')

2024/01/26 20:56:42 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/01/26 20:56:43 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/content/mlruns/1', creation_time=1706302603995, experiment_id='1', last_update_time=1706302603995, lifecycle_stage='active', name='Learn MLFLOW', tags={}>

In [7]:
print('ngrok config add-authtoken 2ZrJVVeIYs9WvY2PriDSGfOk8Iz_7FtnWPBGCTht5dRGuJ4jZ')
conf.get_default().auth_token = getpass.getpass()
port=5000
public_url =ngrok.connect(port).public_url
print(f' *ngrok tunnel \'{public_url}\' -> \'http://127.0.0.1:{port}\'')

ngrok config add-authtoken 2ZrJVVeIYs9WvY2PriDSGfOk8Iz_7FtnWPBGCTht5dRGuJ4jZ
··········
 *ngrok tunnel 'https://c5db-34-86-21-92.ngrok-free.app' -> 'http://127.0.0.1:5000'


In [8]:
def read_dataframe(filename):
  lab = pd.read_parquet(filename)

  lab.lpep_pickup_datetime = pd.to_datetime(lab.lpep_pickup_datetime)    #datetime - to tell pandas that it is not a string but date
  lab.lpep_dropoff_datetime = pd.to_datetime(lab.lpep_dropoff_datetime)

  lab['duration'] = lab.lpep_dropoff_datetime - lab.lpep_pickup_datetime   # adding duration to the list
  lab.duration = lab.duration.apply(lambda td: td.total_seconds() / 60)

  lab = lab[((lab.duration >= 1) & (lab.duration <= 60))]

  categ = ['PULocationID', 'DOLocationID']
  lab[categ] = lab[categ].astype(str)

  return lab

In [10]:
from google.colab import files
upload = files.upload()

Saving green_tripdata_2021-02.parquet to green_tripdata_2021-02.parquet


In [11]:
lab_train = read_dataframe('green_tripdata_2021-01.parquet')
lab_val = read_dataframe('green_tripdata_2021-02.parquet')

In [12]:
len(lab_train), len(lab_val)

(73908, 61921)

In [13]:
lab_train['PU_DO'] = lab_train['PULocationID'].astype(str) + '_' + lab_train['DOLocationID'].astype(str)
lab_val['PU_DO'] = lab_val['PULocationID'].astype(str) + '_' + lab_val['DOLocationID'].astype(str)

In [14]:
categ = ['PU_DO']#, 'DOLocationID']
num = ['trip_distance']

dv =DictVectorizer()    # turns a dictionary into a vector

train_dict = lab_train[categ + num].to_dict(orient='records') #iloc[:10 - takes first 10,  this syntax turns the dataframe into dictionaries
x_train = dv.fit_transform(train_dict)

val_dicts = lab_val[categ + num].to_dict(orient='records')
x_val =  dv.transform(val_dicts)

In [15]:
target = 'duration'
y_train = lab_train[target].values
y_val = lab_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_val)

mean_squared_error(y_val, y_pred, squared=False)

In [17]:
# Creating the 'models' directory because it doesn't exist
if not os.path.exists('models'):
    os.makedirs('models')

# original code to save the file
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)


In [18]:
# End the current active run
mlflow.end_run()

# Start a new run
with mlflow.start_run():
    mlflow.set_tag('developer', 'Victoria')

    mlflow.log_param('train-data-path', 'green_tripdata_2021-01.parquet')
    mlflow.log_param('val-data-path', 'green_tripdata_2021-02.parquet')

    alpha = 0.01
    mlflow.log_param('alpha', alpha)

    lr = Lasso(alpha)
    lr.fit(x_train, y_train)

    y_pred = lr.predict(x_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

    mlflow.log_artifact(local_path='models/lin_reg.bin', artifact_path='models_pickle/lin_reg.bin')  #saved the model in lin_reg.bin in the artifact directory


In [19]:
train = xgb.DMatrix(x_train, label=y_train)
valid = xgb.DMatrix(x_val, label=y_val)

In [20]:
def objective(params):
      #  params = log and visualize parameters using MLflow
  with mlflow.start_run():
     mlflow.set_tag('model', 'xgboost')
     mlflow.log_params(params)
     booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
     )
     y_pred = booster.predict(valid)
     rmse = mean_squared_error(y_val, y_pred, squared=False)
     mlflow.log_metric('rmse', rmse)

  return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
 search_space = {
     'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
     'learning_rate': hp.loguniform('learning_rate', -3, 0),
     'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
     'reg_lambda': hp.loguniform('reg_lamba', -6, -1),
     'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
     'objective': 'reg:linear',
     'seed': 42,
 }

 best_result = fmin(
     fn=objective,
     space=search_space,
     algo=tpe.suggest,
     max_evals=50,
     trials=Trials())

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)